In [1]:
from bs4 import BeautifulSoup
import os
import requests
import numpy as np
import pandas as pd

In [2]:
url = 'https://ben10.fandom.com/wiki/Category:Villains'
response = requests.get(url)
page_contents = response.text
doc = BeautifulSoup(page_contents, 'html.parser')

In [3]:
li_tags = doc.find_all('li', {'class': 'category-page__member'})

In [4]:
# Getting title names
titles = []
for i in range(1,len(li_tags)):
    div_tag = li_tags[i].find('div')
    if div_tag and div_tag.a and div_tag.a.get('title'):
        titles.append(div_tag.a.get('title'))
    elif li_tags[i].a and li_tags[i].a.get('title'):
        titles.append(li_tags[i].a.get('title'))


In [6]:
len(titles)

199

In [7]:
vallain_urls = []
for i in range(1,len(li_tags)):
    try:
        vallain_urls.append("https://ben10.fandom.com/" + li_tags[i].a.get('href'))
    except requests.exceptions.RequestException as e:
        print(f"Failed to get alien data: {e}")
        vallain_urls = []

In [9]:
len(vallain_urls)

199

In [10]:
vallain_urls[5]

'https://ben10.fandom.com//wiki/Albedo'

In [12]:
alien_page_url = vallain_urls[0]
response = requests.get(alien_page_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print("Request successful!")
else:
    print(f"Request failed with status code {response.status_code}")


Request successful!


### Image url scrapping

In [14]:
images = []
for i in range(len(vallain_urls)):
    print(i)
    alien_page_url = vallain_urls[i]
    response = requests.get(alien_page_url)
    alien_doc = BeautifulSoup(response.text, 'html.parser')
    try:
        section = alien_doc.find('figure', class_='pi-item pi-image')
        if section:
            img_url = section.a.get('href')
            images.append(img_url)
        else:
            images.append('No Image')
        
        
    except Exception as e:
        print(titles[i],e)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198


In [20]:
len(images)

199

In [34]:
images.count('No Image')

20

### Information scrapping!

In [22]:
basic = ['Species','Home World','Body','Abilities','First Appearance']
species = []
home_world = []
body = []
abilities=[]
first_appearance = []
for i in range(len(vallain_urls)):
    alien_page_url = vallain_urls[i]
    response = requests.get(alien_page_url)
    alien_doc = BeautifulSoup(response.text, 'html.parser')
    basic_info = ['Species','Home World','Body','Abilities','First Appearance']
    h3_tags = alien_doc.find_all('h3',{'class':'pi-data-label pi-secondary-font'})
    keys = [tag.text for tag in h3_tags]
    div_tags = alien_doc.find_all('div',{'class':'pi-data-value pi-font'})
    
    values = []
    for tag in div_tags:
    # Replace <br/> tags with a comma
        for br_tag in tag.find_all('<br/>'):
            br_tag.replace_with(',')
    
        text_content = tag.get_text(separator=',', strip=True)
        values.append(text_content)
    values = [element.split('[')[0] if '[' in element else element for element in values]
    
    species.append(values[keys.index('Species')] if 'Species' in keys else 'No info')
    home_world.append(values[keys.index('Home World')] if 'Home World' in keys else 'No info')
    body.append(values[keys.index('Body')] if 'Body' in keys else 'No info')
    abilities.append(values[keys.index('Abilities')] if 'Abilities' in keys else 'No info')
    first_appearance.append(values[keys.index('First Appearance')] if 'First Appearance' in keys else 'No info')
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198


In [23]:
len(species)

199

In [24]:
species.count('No info')

61

In [25]:
len(home_world)

199

In [26]:
home_world.count('No info')

87

In [27]:
len(body)

199

In [28]:
body.count('No info')

176

In [29]:
len(abilities)

199

In [30]:
abilities.count('No info')

84

In [31]:
len(first_appearance)

199

In [32]:
first_appearance.count('No info')

21

In [35]:
villains_data = pd.DataFrame({
    'title': titles,
    'species':species,
    'home_world':home_world,
    'body':body,
    'abilities':abilities,
    'first_appearance':first_appearance,
    'img_links':images
})

In [36]:
villains_data['character_type'] = 'Villain'

In [37]:
villains_data.head()

,title,species,home_world,body,abilities,first_appearance,img_links,character_type
0,Acid Breath (Classic),"Mutant,Human",Earth,No info,"Acid Breath,Acid Spit,Acid Ignition,Acid Immun...",Last Laugh,https://static.wikia.nocookie.net/ben10/images...,Villain
1,Acid Breath (Reboot),"Mutant,Human",Earth,No info,"Acid Spit,Enhanced Agility",Zombozo-Land,https://static.wikia.nocookie.net/ben10/images...,Villain
2,Addwaitya,"Altered,Geochelone Aerio,",Aldabra,No info,"Magic,Mana,Manipulation,Life-Force Generation,...",Where the Magic Happens,https://static.wikia.nocookie.net/ben10/images...,Villain
3,Aggrebots,No info,No info,No info,No info,Escape from Aggregor,https://static.wikia.nocookie.net/ben10/images...,Villain
4,Aggregor,"Artificial Mutant,Human,(,Osmosian,),",No info,No info,"Enhanced Spearmanship,Expert Strategist,Keen T...",Fame,https://static.wikia.nocookie.net/ben10/images...,Villain


In [38]:
villains_data.shape

(199, 8)

In [39]:
villains_data.to_csv('D:\\Study\\ML-Projects\\Project-3(Recommending_system)\\data\\villains.csv',index=False)